In [98]:
import numpy as np
import math
import os
import sys
import progressbar

dimension = 39

In [99]:
def get_confusion_mat(true_class, pred_class):
    conf_mat = np.full((3,3),1e-10)
    for i in range(len(true_class)):
        conf_mat[true_class[i]][pred_class[i]]+=1
    return np.array(conf_mat)

In [100]:
def get_metric(conf_matrix):
    x = conf_matrix.shape[0]
    precision = []
    accuracy = []
    recall = []
    f_measure = []
    total_sum = np.sum(conf_matrix)
    for i in range(x):
        TP = conf_matrix[i][i]
        FN = np.sum(conf_matrix[i,:])-conf_matrix[i][i]
        FP = np.sum(conf_matrix[:,i])-conf_matrix[i][i]
        TN = total_sum-(TP+FP+FN)

        recal = TP/(TP+FN)
        preci = TP/(TP+FP)
        acc = (TP+TN)/(TP+TN+FP+FN)
        
        precision.append(preci)
        recall.append(recal)
        accuracy.append(acc)
        f_measure.append((2*recal*preci)/(recal+preci))
        
    return accuracy,precision,recall,f_measure

In [101]:
def find_distance(data1, data2):
    su = 0.0
    for i in range(dimension):
        su += ((data1[i]-data2[i])**2);
    return su**0.5
        
    

In [102]:
def dtw(data1, data2):
    
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    
    dp = np.ndarray(shape=(n1,n2),dtype=float)
    dp[0][0]=find_distance(data1[0,:],data2[0,:])
    for i in range(1,n1):
        dp[i][0] = find_distance(data1[i,:], data2[0,:])+dp[i-1][0]
    for i in range(1,n2):
        dp[0][i] = find_distance(data1[0,:], data2[i,:])+dp[0][i-1]
        
    for i in range(1,n1):
        for j in range(1,n2):
            dp[i][j] = find_distance(data1[i,:],data2[j,:]) + min(dp[i-1][j],dp[i-1][j-1],dp[i][j-1])
    
    return dp[n1-1][n2-1]/(n1*n2)
            

In [103]:
def get_predictions(path, st,k):
    pred_class = []
    for f in progressbar.progressbar(os.listdir(os.path.join(path,st))):
        data_test = np.load(os.path.join(os.path.join(path,st),f))
        lis = []
        for f_train in os.listdir(os.path.join('Group13/Train','di_npy')):
            data_train = np.load(os.path.join(os.path.join('Group13/Train','di_npy'),f_train))
            dist = dtw(data_train, data_test)
            dist = dist
            lis.append([dist,0])
        for f_train in os.listdir(os.path.join('Group13/Train','dI_npy')):
            data_train = np.load(os.path.join(os.path.join('Group13/Train','dI_npy'),f_train))
            dist = dtw(data_train, data_test)
            dist = dist
            lis.append([dist,1])
        for f_train in os.listdir(os.path.join('Group13/Train','DI_npy')):
            data_train = np.load(os.path.join(os.path.join('Group13/Train','DI_npy'),f_train))
            dist = dtw(data_train, data_test)
            dist = dist
            lis.append([dist,2])
        lis2 = sorted(lis,key=lambda x: x[0])
        coun = [0,0,0]
        for i in range(k):
            coun[lis2[i][1]]+=1
        pred_class.append(np.argmax(coun))
        
    return pred_class
    

In [104]:
def main():
    K = [4, 8, 16, 32]
    path = 'Group13/Test'
    true_class = []
    for f in os.listdir(os.path.join(path,'di_npy')):
        true_class.append(0)
    for f in os.listdir(os.path.join(path,'dI_npy')):
        true_class.append(1)
    for f in os.listdir(os.path.join(path,'DI_npy')):
        true_class.append(2)
        
    for k in K:
        pred_class = []
        p_class1 = get_predictions(path,'di_npy',k)
        p_class2 = get_predictions(path,'dI_npy',k)
        p_class3 = get_predictions(path,'DI_npy',k)
        for i in range(len(p_class1)):
            pred_class.append(p_class1[i])
        for i in range(len(p_class2)):
            pred_class.append(p_class2[i])
        for i in range(len(p_class3)):
            pred_class.append(p_class3[i])
        
        
        conf_mat = get_confusion_mat(true_class, pred_class)
        accuracy,precision,recall,f_measure = get_metric(conf_mat)
        print('#'*10,"Done for K: {}".format(k),'#'*10)
        print(conf_mat)
        print("Accuracy:\t ", round(accuracy[0],4), round(accuracy[1],4), round(accuracy[2],4))
        print("Precision:\t ",round(precision[0],4), round(precision[1],4), round(precision[2],4))
        print("Recall:\t ",   round(recall[0],4), round(recall[1],4), round(recall[2],2))
        print("F_measure:\t ",round(f_measure[0],4), round(f_measure[1],4), round(f_measure[2],4))
        
        

In [105]:
if(__name__=="__main__"):
    main()

100% (27 of 27) |########################| Elapsed Time: 0:01:17 Time:  0:01:17
100% (24 of 24) |########################| Elapsed Time: 0:01:30 Time:  0:01:30
100% (14 of 14) |########################| Elapsed Time: 0:00:42 Time:  0:00:42
N/A% (0 of 27) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

########## Done for K: 4 ##########
[[ 13.  13.   1.]
 [  1.  19.   4.]
 [  1.   9.   4.]]
Accuracy:	  0.7538 0.5846 0.7692
Precision:	  0.8667 0.4634 0.4444
Recall:	  0.4815 0.7917 0.29
F_measure:	  0.619 0.5846 0.3478


100% (27 of 27) |########################| Elapsed Time: 0:01:14 Time:  0:01:14
100% (24 of 24) |########################| Elapsed Time: 0:01:28 Time:  0:01:28
100% (14 of 14) |########################| Elapsed Time: 0:00:39 Time:  0:00:39
N/A% (0 of 27) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

########## Done for K: 8 ##########
[[  1.30000000e+01   1.40000000e+01   1.00000000e-10]
 [  1.00000000e+00   2.10000000e+01   2.00000000e+00]
 [  1.00000000e-10   9.00000000e+00   5.00000000e+00]]
Accuracy:	  0.7692 0.6 0.8308
Precision:	  0.9286 0.4773 0.7143
Recall:	  0.4815 0.875 0.36
F_measure:	  0.6341 0.6176 0.4762


100% (27 of 27) |########################| Elapsed Time: 0:01:11 Time:  0:01:11
100% (24 of 24) |########################| Elapsed Time: 0:01:24 Time:  0:01:24
100% (14 of 14) |########################| Elapsed Time: 0:00:40 Time:  0:00:40
N/A% (0 of 27) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

########## Done for K: 16 ##########
[[  1.40000000e+01   1.30000000e+01   1.00000000e-10]
 [  3.00000000e+00   2.00000000e+01   1.00000000e+00]
 [  1.00000000e-10   1.00000000e+01   4.00000000e+00]]
Accuracy:	  0.7538 0.5846 0.8308
Precision:	  0.8235 0.4651 0.8
Recall:	  0.5185 0.8333 0.29
F_measure:	  0.6364 0.597 0.4211


100% (27 of 27) |########################| Elapsed Time: 0:01:14 Time:  0:01:14
100% (24 of 24) |########################| Elapsed Time: 0:01:24 Time:  0:01:24
100% (14 of 14) |########################| Elapsed Time: 0:00:42 Time:  0:00:42


########## Done for K: 32 ##########
[[  1.10000000e+01   1.60000000e+01   1.00000000e-10]
 [  1.00000000e+00   2.20000000e+01   1.00000000e+00]
 [  1.00000000e-10   9.00000000e+00   5.00000000e+00]]
Accuracy:	  0.7385 0.5846 0.8462
Precision:	  0.9167 0.4681 0.8333
Recall:	  0.4074 0.9167 0.36
F_measure:	  0.5641 0.6197 0.5


In [85]:
x1 = [[-4.2,1.0],[-3.2,1.6],[-2.0,2.0],[-1.2,1.85],[0.0,2.0],[1.2,2.6],[2.0,3.2],[3.1,2.75],[4.0,2.1]]
x2 = [[-2.2,0.5],[-1.2,0.6],[0.0,1.0],[1.2,1.2],[2.0,1.4],[3.2,1.2]]
x1 = np.array(x1)
x2 = np.array(x2)
print(x1.shape)
print(dtw(x1,x2))


(9, 2)
[[  2.06155281   5.088102     9.288102    14.69180444  20.90469426
   28.30739647]
 [  3.54815969   4.29762079   7.55338491  11.97152932  17.17537405
   23.58786187]
 [  5.06143428   5.16061124   6.53368877   9.83217327  13.87692295
   19.13810153]
 [  6.74146404   6.31143428   6.63115535   9.02015212  12.25163785
   16.69939009]
 [  9.40416944   8.15534317   7.31143428   8.07337586  10.16143717
   13.45992167]
 [ 13.40041768  11.27944304   9.31143428   8.71143428   9.51559637
   11.9569075 ]
 [ 18.39341277  15.40254867  12.28464803  10.86550021  10.51143428
   11.84797713]
 [ 24.15123354  20.21009482  15.84449353  13.31754017  12.25284176
   12.06465674]
 [ 30.55435777  25.62211847  19.99298724  16.25862841  14.37180377
   13.2688162 ]]
13.2688161972
